In [4]:
import pandas as pd
import utils
import plotly.graph_objects as go

In [5]:
def plot_candles(df_plot):
    fig = go.Figure()

    fig.add_trace(go.Candlestick(
        x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
        line=dict(width=1), opacity=1,
        increasing_fillcolor='#24A06B',
        decreasing_fillcolor="#CC2E3C",
        increasing_line_color='#2EC886',  
        decreasing_line_color='#FF3A4C'
    ))

    for c in ['MA_TP', 'BB_UPPER', 'BB_LOWER']:
        fig.add_trace(go.Scatter( 
            x=df_plot.time,
            y=df_plot[c],
            line=dict(width=2,color="#4290f5"),
            line_shape='spline',
            name=c,
            mode='lines'
            ))        

    df_markers = df_plot[df_plot.BB_SIGNAL!=0]
    fig.add_trace(go.Candlestick(
        x=df_markers.time, open=df_markers.mid_o, high=df_markers.mid_h, low=df_markers.mid_l, close=df_markers.mid_c,
        line=dict(width=1), opacity=1,
        increasing_fillcolor='#3480eb',
        decreasing_fillcolor="#3480eb",
        increasing_line_color='#3480eb',  
        decreasing_line_color='#3480eb'
    ))

    fig.update_layout(width=1000,height=400,
        margin=dict(l=10,r=10,b=10,t=10),
        font=dict(size=10,color="#e1e1e1"),
        paper_bgcolor="#1e1e1e",
        plot_bgcolor="#1e1e1e")
    fig.update_xaxes(
        gridcolor="#1f292f",
        showgrid=True,fixedrange=True,rangeslider=dict(visible=False),
        rangebreaks=[
            dict(bounds=["sat", "mon"])
        ]
    )
    fig.update_yaxes(
        gridcolor="#1f292f",
        showgrid=True
    )
    fig.show()

In [6]:
pair = "GBP_USD"
granularity = "M5"
df_raw = pd.read_pickle(utils.get_his_data_filename(pair, granularity))

In [7]:
cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']

In [8]:
def Get_BB_Signal(row):
    if row.mid_c > row.BB_UPPER and row.mid_c > row.mid_o:
        return -1
    if row.mid_c < row.BB_LOWER and row.mid_c < row.mid_o:
        return 1
    return 0

In [9]:
df = df_raw[cols].copy()
df['TP'] = (df.mid_h + df.mid_l + df.mid_c) / 3
df['MA_TP'] = df.TP.rolling(window=20).mean()
df['STD_TP'] = df.TP.rolling(window=20).std()
df['BB_UPPER'] = df.MA_TP + 2 * df.STD_TP
df['BB_LOWER'] = df.MA_TP - 2 * df.STD_TP
df['BB_SIGNAL'] = df.apply(Get_BB_Signal, axis=1)

In [10]:
df_plot = df.iloc[1000:1150]
plot_candles(df_plot)

In [11]:
df['EMA_LONG'] = df.mid_c.ewm(span=26, min_periods=26).mean()
df['EMA_SHORT'] = df.mid_c.ewm(span=12, min_periods=12).mean()
df['MACD'] = df.EMA_SHORT - df.EMA_LONG
df['SIGNAL'] = df.MACD.ewm(span=9).mean()
df['HIST'] = df.MACD - df.SIGNAL
df.dropna(inplace=True)

In [12]:
df.head()

,time,mid_o,mid_h,mid_l,mid_c,TP,MA_TP,STD_TP,BB_UPPER,BB_LOWER,BB_SIGNAL,EMA_LONG,EMA_SHORT,MACD,SIGNAL,HIST
25,2018-01-02 00:05:00+00:00,1.35116,1.35138,1.35114,1.35120,1.351240,1.351426,0.000276,1.351978,1.350874,0,1.351403,1.351342,-0.000061,-0.000061,0.000000
26,2018-01-02 00:10:00+00:00,1.35112,1.35116,1.35102,1.35114,1.351107,1.351417,0.000284,1.351984,1.350849,0,1.351381,1.351311,-0.000070,-0.000066,-0.000004
27,2018-01-02 00:15:00+00:00,1.35115,1.35139,1.35114,1.35135,1.351293,1.351420,0.000282,1.351984,1.350856,0,1.351378,1.351317,-0.000061,-0.000064,0.000003
28,2018-01-02 00:20:00+00:00,1.35137,1.35162,1.35122,1.35142,1.351420,1.351438,0.000269,1.351976,1.350900,0,1.351382,1.351333,-0.000049,-0.000059,0.000010
29,2018-01-02 00:25:00+00:00,1.35148,1.35150,1.35124,1.35146,1.351400,1.351442,0.000268,1.351978,1.350906,0,1.351388,1.351353,-0.000035,-0.000052,0.000016


In [15]:
df_plot = df.iloc[100:200]
fig = go.Figure()

fig.add_trace(go.Scatter( 
    x=df_plot.time,
    y=df_plot.SIGNAL,
    line=dict(width=2,color="#03adfc"),
    line_shape='spline',
    name='SIGNAL',
    mode='lines'
    ))        
fig.add_trace(go.Scatter( 
    x=df_plot.time,
    y=df_plot.MACD,
    line=dict(width=2,color="#fc4a03"),
    line_shape='spline',
    name='MACD',
    mode='lines'
    ))         
fig.add_trace(go.Bar( 
    x=df_plot.time,
    y=df_plot.HIST,
    name='HIST'
    ))         

fig.update_layout(width=1000,height=400,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False),
    rangebreaks=[
        dict(bounds=["sat", "mon"])
    ]
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()